In [1]:
import pandas as pd
from datetime import datetime


# Load the cleaned dataset
df = pd.read_csv('/home/abdou-salam/Desktop/us-regional-sales-analysis/data/processed/fully_cleaned_us_regional_sales_data.csv')

In [2]:
print("="*80)
print("COMPREHENSIVE SALES PERFORMANCE & PRODUCT INTELLIGENCE ANALYSIS")
print("="*80)

# Ensure date columns are properly formatted
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')
df['Order Quarter'] = df['OrderDate'].dt.to_period('Q')
df['Order Month'] = df['OrderDate'].dt.month
df['Order Year'] = df['OrderDate'].dt.year

''' SECTION 1: ENHANCED SALES PERFORMANCE ANALYSIS '''
print("\n1. COMPREHENSIVE SALES PERFORMANCE BY CHANNEL")
print("-" * 60)

sales_performance_by_channel = df.groupby('Sales Channel').agg({
    'Order Quantity': ['sum', 'mean', 'count'],
    'Total Revenue': ['sum', 'mean', 'std'],
    'Total Profit': ['sum', 'mean'],
    'Profit Margin': ['mean', 'std'],
    'Unit Price': 'mean',
    'Discount Applied': 'mean',
    'OrderNumber': 'nunique'
}).round(2)

sales_performance_by_channel.columns = ['_'.join(col).strip() for col in sales_performance_by_channel.columns]
print("Enhanced Sales Performance by Channel:")
print(sales_performance_by_channel)

# Channel efficiency metrics
channel_efficiency = df.groupby('Sales Channel').agg({
    'Total Revenue': 'sum',
    'OrderNumber': 'nunique',
    'Order Quantity': 'sum'
}).round(2)

channel_efficiency['Revenue_per_Order'] = (channel_efficiency['Total Revenue'] / channel_efficiency['OrderNumber']).round(2)
channel_efficiency['Revenue_per_Unit'] = (channel_efficiency['Total Revenue'] / channel_efficiency['Order Quantity']).round(2)

print("\nChannel Efficiency Metrics:")
print(channel_efficiency)

COMPREHENSIVE SALES PERFORMANCE & PRODUCT INTELLIGENCE ANALYSIS

1. COMPREHENSIVE SALES PERFORMANCE BY CHANNEL
------------------------------------------------------------
Enhanced Sales Performance by Channel:
               Order Quantity_sum  Order Quantity_mean  Order Quantity_count  \
Sales Channel                                                                  
Distributor                  6287                 4.57                  1375   
In-Store                    14878                 4.51                  3298   
Online                      10897                 4.49                  2425   
Wholesale                    4100                 4.59                   893   

               Total Revenue_sum  Total Revenue_mean  Total Revenue_std  \
Sales Channel                                                             
Distributor          13149207.07             9563.06            9545.45   
In-Store             30059022.91             9114.32            8848.49   
Online  

In [3]:
''' SECTION 2: ADVANCED REVENUE ANALYSIS '''
print("\n" + "="*80)
print("ADVANCED REVENUE & MARKET ANALYSIS")
print("="*80)

print("\n2. REVENUE DISTRIBUTION ANALYSIS")
print("-" * 60)

# Enhanced revenue analysis
revenue_analysis = df.groupby('Sales Channel').agg({
    'Total Revenue': ['sum', 'mean', 'median', 'std', 'min', 'max'],
    'Order Quantity': 'sum',
    'OrderNumber': 'count'
}).round(2)

revenue_analysis.columns = ['_'.join(col).strip() for col in revenue_analysis.columns]

# Market share and concentration
total_revenue = df['Total Revenue'].sum()
channel_market_share = df.groupby('Sales Channel')['Total Revenue'].sum()
channel_market_share_pct = (channel_market_share / total_revenue * 100).round(2)

# Calculate Herfindahl-Hirschman Index (market concentration)
hhi = ((channel_market_share_pct / 100) ** 2).sum()
market_concentration = "High" if hhi > 0.25 else "Moderate" if hhi > 0.15 else "Low"

print("Revenue Distribution by Channel:")
print(revenue_analysis)
print(f"\nMarket Share by Channel (%):")
print(channel_market_share_pct.sort_values(ascending=False))
print(f"\nMarket Concentration Index (HHI): {hhi:.3f} - {market_concentration} Concentration")


ADVANCED REVENUE & MARKET ANALYSIS

2. REVENUE DISTRIBUTION ANALYSIS
------------------------------------------------------------
Revenue Distribution by Channel:
               Total Revenue_sum  Total Revenue_mean  Total Revenue_median  \
Sales Channel                                                                
Distributor          13149207.07             9563.06               6287.28   
In-Store             30059022.91             9114.32               6041.39   
Online               21667833.90             8935.19               6042.06   
Wholesale             8160520.34             9138.32               6289.29   

               Total Revenue_std  Total Revenue_min  Total Revenue_max  \
Sales Channel                                                            
Distributor              9545.45             134.00           49035.96   
In-Store                 8848.49             100.50           49697.92   
Online                   8582.42             100.50           48119.40 

In [4]:
print("\n3. REVENUE VOLATILITY & STABILITY ANALYSIS")
print("-" * 60)

# Revenue volatility by month for each channel
monthly_revenue = df.groupby(['Order Month', 'Sales Channel'])['Total Revenue'].sum().reset_index()
revenue_volatility = monthly_revenue.groupby('Sales Channel')['Total Revenue'].agg(['std', 'mean']).round(2)
revenue_volatility['Coefficient_of_Variation'] = (revenue_volatility['std'] / revenue_volatility['mean'] * 100).round(2)
revenue_volatility = revenue_volatility.sort_values('Coefficient_of_Variation')

print("Revenue Volatility Analysis (Lower CV = More Stable):")
print(revenue_volatility)


3. REVENUE VOLATILITY & STABILITY ANALYSIS
------------------------------------------------------------
Revenue Volatility Analysis (Lower CV = More Stable):
                     std        mean  Coefficient_of_Variation
Sales Channel                                                 
Online         375287.46  1805652.83                     20.78
In-Store       536942.52  2504918.58                     21.44
Distributor    268846.50  1095767.26                     24.54
Wholesale      196099.86   680043.36                     28.84


In [ ]:
''' SECTION 3: PROFITABILITY ANALYSIS '''
print("\n" + "="*80)
print("PROFITABILITY & ROI ANALYSIS")
print("="*80)

print("\n4. COMPREHENSIVE PROFITABILITY METRICS")
print("-" * 60)

profitability_analysis = df.groupby('Sales Channel').agg({
    'Total Profit': ['sum', 'mean', 'std'],
    'Total Revenue': 'sum',
    'Total Cost': 'sum',
    'Profit Margin': ['mean', 'median', 'std', 'min', 'max'],
    'Unit Cost': 'mean',
    'Unit Price': 'mean'
}).round(2)

profitability_analysis.columns = ['_'.join(col).strip() for col in profitability_analysis.columns]

# Calculate additional profitability metrics
channel_profitability = df.groupby('Sales Channel').agg({
    'Total Profit': 'sum',
    'Total Revenue': 'sum',
    'Total Cost': 'sum',
    'OrderNumber': 'nunique'
})

channel_profitability['ROI_Percent'] = (channel_profitability['Total Profit'] / channel_profitability['Total Cost'] * 100).round(2)
channel_profitability['Profit_per_Order'] = (channel_profitability['Total Profit'] / channel_profitability['OrderNumber']).round(2)
channel_profitability['Revenue_to_Cost_Ratio'] = (channel_profitability['Total Revenue'] / channel_profitability['Total Cost']).round(2)

print("Detailed Profitability Analysis:")
print(profitability_analysis)
print(f"\nAdvanced Profitability Metrics:")
print(channel_profitability[['ROI_Percent', 'Profit_per_Order', 'Revenue_to_Cost_Ratio']])



ADVANCED PROFITABILITY & ROI ANALYSIS

4. COMPREHENSIVE PROFITABILITY METRICS
------------------------------------------------------------
Detailed Profitability Analysis:
               Total Profit_sum  Total Profit_mean  Total Profit_std  \
Sales Channel                                                          
Distributor          3867956.81            2813.06           3578.21   
In-Store             8753971.45            2654.33           3399.04   
Online               6136090.74            2530.35           3224.02   
Wholesale            2460496.34            2755.32           3601.95   

               Total Revenue_sum  Total Cost_sum  Profit Margin_mean  \
Sales Channel                                                          
Distributor          13149207.07      9281250.26               28.67   
In-Store             30059022.91     21305051.46               28.34   
Online               21667833.90     15531743.16               28.04   
Wholesale             8160520.34  

In [6]:
''' SECTION 4: PRODUCT INTELLIGENCE ANALYSIS '''
print("\n" + "="*80)
print("PRODUCT INTELLIGENCE & PORTFOLIO ANALYSIS")
print("="*80)

print("\n5. PRODUCT PORTFOLIO PERFORMANCE")
print("-" * 60)

# Enhanced product analysis
product_performance = df.groupby('_ProductID').agg({
    'Total Revenue': ['sum', 'mean', 'count'],
    'Total Profit': 'sum',
    'Order Quantity': 'sum',
    'Profit Margin': 'mean',
    'Discount Applied': 'mean',
    'Unit Price': 'mean'
}).round(2)

product_performance.columns = ['_'.join(col).strip() for col in product_performance.columns]

# Product classification using BCG Matrix approach
product_metrics = df.groupby('_ProductID').agg({
    'Total Revenue': 'sum',
    'Order Quantity': 'sum',
    'OrderNumber': 'count',
    'Profit Margin': 'mean'
})

# Market share and growth proxies
product_metrics['Market_Share'] = product_metrics['Total Revenue'] / product_metrics['Total Revenue'].sum() * 100
product_metrics['Order_Frequency'] = product_metrics['OrderNumber']
product_metrics['Revenue_per_Unit'] = product_metrics['Total Revenue'] / product_metrics['Order Quantity']


# Classify products (simplified BCG matrix)
market_share_median = product_metrics['Market_Share'].median()
order_freq_median = product_metrics['Order_Frequency'].median()

def classify_product(row):
    if row['Market_Share'] > market_share_median and row['Order_Frequency'] > order_freq_median:
        return "Star Products"
    elif row['Market_Share'] > market_share_median and row['Order_Frequency'] <= order_freq_median:
        return "Cash Cow"
    elif row['Market_Share'] <= market_share_median and row['Order_Frequency'] > order_freq_median:
        return "Question Mark"
    else:
        return "Dog Products"

product_metrics['Product_Category'] = product_metrics.apply(classify_product, axis=1)

# Product sales share
product_sales_share = df.groupby('_ProductID')['Total Revenue'].sum()
product_sales_share_pct = (product_sales_share / product_sales_share.sum() * 100).sort_values(ascending=False)

print("Top 15 Products by Sales Share (% of Total Revenue):")
print(product_sales_share_pct.head(15).round(3))

print("\nProduct Portfolio Classification:")
portfolio_summary = product_metrics.groupby('Product_Category').agg({
    'Total Revenue': ['count', 'sum'],
    'Market_Share': 'mean',
    'Profit Margin': 'mean'
}).round(2)
portfolio_summary.columns = ['Product_Count', 'Total_Revenue', 'Avg_Market_Share', 'Avg_Profit_Margin']
print(portfolio_summary)



PRODUCT INTELLIGENCE & PORTFOLIO ANALYSIS

5. PRODUCT PORTFOLIO PERFORMANCE
------------------------------------------------------------
Top 15 Products by Sales Share (% of Total Revenue):
_ProductID
23    2.873
40    2.553
37    2.501
41    2.474
5     2.468
4     2.445
2     2.428
35    2.414
8     2.352
17    2.344
29    2.305
14    2.291
16    2.288
38    2.283
10    2.257
Name: Total Revenue, dtype: float64

Product Portfolio Classification:
                  Product_Count  Total_Revenue  Avg_Market_Share  \
Product_Category                                                   
Cash Cow                      6     9895852.97              2.26   
Dog Products                 20    27609664.11              1.89   
Question Mark                 4     6042025.29              2.07   
Star Products                17    29489041.85              2.38   

                  Avg_Profit_Margin  
Product_Category                     
Cash Cow                      28.59  
Dog Products            

In [7]:
print("\n6. PRODUCT-CHANNEL PERFORMANCE MATRIX")
print("-" * 60)

# Product performance by channel
product_channel_matrix = pd.pivot_table(df, 
                                       values='Total Revenue', 
                                       index='_ProductID', 
                                       columns='Sales Channel', 
                                       aggfunc='sum', 
                                       fill_value=0).round(2)

# Top products by channel
print("Top 5 Products by Revenue for Each Channel:")
for channel in df['Sales Channel'].unique():
    channel_products = df[df['Sales Channel'] == channel].groupby('_ProductID')['Total Revenue'].sum().sort_values(ascending=False).head(5)
    print(f"\n{channel}:")
    print(channel_products.round(2))


6. PRODUCT-CHANNEL PERFORMANCE MATRIX
------------------------------------------------------------
Top 5 Products by Revenue for Each Channel:

In-Store:
_ProductID
23    1050335.28
17     859810.40
4      825563.82
40     824385.35
37     754819.19
Name: Total Revenue, dtype: float64

Online:
_ProductID
12    660349.05
23    603808.22
5     597848.91
15    581528.44
18    577852.02
Name: Total Revenue, dtype: float64

Distributor:
_ProductID
41    438858.71
25    435768.40
35    433584.00
5     413568.42
38    399101.78
Name: Total Revenue, dtype: float64

Wholesale:
_ProductID
37    286055.09
26    259112.85
4     258482.52
5     255388.86
14    254517.05
Name: Total Revenue, dtype: float64


In [8]:
print("\n7. ADVANCED PRODUCT INSIGHTS")
print("-" * 60)

# Product discount sensitivity analysis
discount_sensitivity = df.groupby('_ProductID').agg({
    'Discount Applied': ['mean', 'std', 'count'],
    'Total Revenue': 'sum',
    'Order Quantity': 'sum'
}).round(3)

discount_sensitivity.columns = ['Avg_Discount', 'Discount_Volatility', 'Discount_Frequency', 'Revenue', 'Quantity']
discount_sensitivity['Revenue_per_Discount_Point'] = (discount_sensitivity['Revenue'] / (discount_sensitivity['Avg_Discount'] * 100 + 1)).round(2)

print("Top 10 Products by Discount Sensitivity (High Revenue per Discount Point):")
print(discount_sensitivity.sort_values('Revenue_per_Discount_Point', ascending=False).head(10)['Revenue_per_Discount_Point'])

# Product lifecycle analysis
product_lifecycle = df.groupby(['_ProductID', 'Order Year'])['Total Revenue'].sum().reset_index()
product_growth = product_lifecycle.pivot(index='_ProductID', columns='Order Year', values='Total Revenue').fillna(0)

# Calculate growth rates if multiple years exist
if product_growth.shape[1] > 1:
    years = sorted(product_growth.columns)
    if len(years) >= 2:
        growth_cols = []
        for i in range(1, len(years)):
            col_name = f'Growth_{years[i-1]}_{years[i]}'
            product_growth[col_name] = ((product_growth[years[i]] - product_growth[years[i-1]]) / 
                                       (product_growth[years[i-1]] + 1) * 100).round(2)
            growth_cols.append(col_name)
        
        if growth_cols:
            print(f"\nProduct Growth Analysis (YoY %):")
            print("Top 10 Fastest Growing Products:")
            avg_growth = product_growth[growth_cols].mean(axis=1).sort_values(ascending=False)
            print(avg_growth.head(10).round(2))



7. ADVANCED PRODUCT INSIGHTS
------------------------------------------------------------
Top 10 Products by Discount Sensitivity (High Revenue per Discount Point):
_ProductID
23    170599.36
5     159533.61
37    156149.23
40    146816.78
41    144551.40
38    141286.45
16    140455.44
2     139615.22
35    137723.10
17    135881.72
Name: Revenue_per_Discount_Point, dtype: float64

Product Growth Analysis (YoY %):
Top 10 Fastest Growing Products:
_ProductID
6     141.08
42     83.94
4      83.24
38     83.06
19     75.32
28     75.30
39     75.16
24     72.16
7      71.20
17     60.66
dtype: float64


In [9]:
''' SECTION 5: TEAM & WAREHOUSE PERFORMANCE '''
print("\n" + "="*80)
print("SALES TEAM & WAREHOUSE PERFORMANCE ANALYSIS")
print("="*80)

print("\n8. SALES TEAM PERFORMANCE ANALYSIS")
print("-" * 60)

team_performance = df.groupby('_SalesTeamID').agg({
    'Total Revenue': ['sum', 'mean', 'count'],
    'Total Profit': 'sum',
    'Order Quantity': 'sum',
    'Profit Margin': 'mean',
    'OrderNumber': 'nunique'
}).round(2)

team_performance.columns = ['_'.join(col).strip() for col in team_performance.columns]

# Team efficiency metrics
team_efficiency = df.groupby('_SalesTeamID').agg({
    'Total Revenue': 'sum',
    'OrderNumber': 'nunique'
})
team_efficiency['Revenue_per_Order'] = (team_efficiency['Total Revenue'] / team_efficiency['OrderNumber']).round(2)

print("Sales Team Performance:")
print(team_performance)
print(f"\nTeam Efficiency (Revenue per Order):")
print(team_efficiency['Revenue_per_Order'].sort_values(ascending=False))


SALES TEAM & WAREHOUSE PERFORMANCE ANALYSIS

8. SALES TEAM PERFORMANCE ANALYSIS
------------------------------------------------------------
Sales Team Performance:
              Total Revenue_sum  Total Revenue_mean  Total Revenue_count  \
_SalesTeamID                                                               
1                    2822397.18             9345.69                  302   
2                    2442319.44             9928.13                  246   
3                    2627627.51             8877.12                  296   
4                    2531754.53             8670.39                  292   
5                    2440924.77             8625.18                  283   
6                    2285274.93             8623.68                  265   
7                    2717742.57             8969.45                  303   
8                    2853026.96             9057.23                  315   
9                    2633670.71             9240.95                  285  

In [10]:
print("\n9. WAREHOUSE PERFORMANCE BY PRODUCT CATEGORIES")
print("-" * 60)

warehouse_performance = df.groupby('WarehouseCode').agg({
    'Total Revenue': ['sum', 'mean'],
    'Order Quantity': 'sum',
    'Total Profit': 'sum',
    'OrderNumber': 'nunique',
    '_ProductID': 'nunique'
}).round(2)

warehouse_performance.columns = ['_'.join(col).strip() for col in warehouse_performance.columns]

# Warehouse specialization index
warehouse_specialization = df.groupby(['WarehouseCode', '_ProductID'])['Total Revenue'].sum().reset_index()
warehouse_diversity = warehouse_specialization.groupby('WarehouseCode')['_ProductID'].count()

print("Warehouse Performance Overview:")
print(warehouse_performance)
print(f"\nWarehouse Product Diversity (Number of Different Products):")
print(warehouse_diversity.sort_values(ascending=False))



9. WAREHOUSE PERFORMANCE BY PRODUCT CATEGORIES
------------------------------------------------------------
Warehouse Performance Overview:
               Total Revenue_sum  Total Revenue_mean  Order Quantity_sum  \
WarehouseCode                                                              
WARE-MKL1006          7800158.36             9101.70                3816   
WARE-NBV1002          6087758.62             8810.07                3101   
WARE-NMK1003         23107373.52             9224.50               11351   
WARE-PUJ1005         13036725.73             8984.65                6572   
WARE-UHY1004         11967354.95             9460.36                5765   
WARE-XYS1001         11037213.04             9032.09                5557   

               Total Profit_sum  OrderNumber_nunique  _ProductID_nunique  
WarehouseCode                                                             
WARE-MKL1006         2334092.56                  857                  47  
WARE-NBV1002         1779

In [ ]:
''' SECTION 6: STRATEGIC INSIGHTS & RECOMMENDATIONS '''
print("\n" + "="*80)
print("STRATEGIC INSIGHTS & BUSINESS RECOMMENDATIONS")
print("="*80)

print("\n10. KEY PERFORMANCE INSIGHTS")
print("-" * 60)

# Generate key insights
top_channel = channel_market_share_pct.index[0]
top_channel_share = channel_market_share_pct.iloc[0]
most_profitable_channel = channel_profitability.sort_values('ROI_Percent', ascending=False).index[0]
highest_roi = channel_profitability.sort_values('ROI_Percent', ascending=False)['ROI_Percent'].iloc[0]

star_products = len(product_metrics[product_metrics['Product_Category'] == 'Star Products'])
total_products = len(product_metrics)
star_percentage = (star_products / total_products * 100)

print(f"""
📊 SALES PERFORMANCE HIGHLIGHTS:

🏆 CHANNEL PERFORMANCE:
   • Market leader: {top_channel} ({top_channel_share:.1f}% market share)
   • Most profitable: {most_profitable_channel} ({highest_roi:.1f}% ROI)
   • Market concentration: {market_concentration} ({hhi:.3f} HHI)

🌟 PRODUCT PORTFOLIO:
   • Total products analyzed: {total_products:,}
   • Star products (high share + frequency): {star_products} ({star_percentage:.1f}%)
   • Portfolio diversification: {"Well-diversified" if star_percentage < 20 else "Concentrated"}

💰 FINANCIAL METRICS:
   • Total revenue analyzed: ${total_revenue:,.0f}
   • Average order value: ${df['Total Revenue'].mean():.2f}
   • Overall profit margin: {df['Profit Margin'].mean():.1f}%
""")



STRATEGIC INSIGHTS & BUSINESS RECOMMENDATIONS

10. KEY PERFORMANCE INSIGHTS
------------------------------------------------------------

📊 SALES PERFORMANCE HIGHLIGHTS:

🏆 CHANNEL PERFORMANCE:
   • Market leader: Distributor (18.0% market share)
   • Most profitable: Wholesale (43.2% ROI)
   • Market concentration: High (0.302 HHI)

🌟 PRODUCT PORTFOLIO:
   • Total products analyzed: 47
   • Star products (high share + frequency): 17 (36.2%)
   • Portfolio diversification: Concentrated

💰 FINANCIAL METRICS:
   • Total revenue analyzed: $73,036,584
   • Average order value: $9139.86
   • Overall profit margin: 28.4%



In [12]:
print("\n11. STRATEGIC RECOMMENDATIONS")
print("-" * 60)

# Generate recommendations based on analysis
most_volatile_channel = revenue_volatility.sort_values('Coefficient_of_Variation', ascending=False).index[0]
most_stable_channel = revenue_volatility.sort_values('Coefficient_of_Variation', ascending=True).index[0]

underperforming_channels = channel_profitability[channel_profitability['ROI_Percent'] < channel_profitability['ROI_Percent'].median()].index.tolist()

print(f"""
🎯 STRATEGIC RECOMMENDATIONS:

CHANNEL OPTIMIZATION:
   • Invest more in {most_profitable_channel} (highest ROI channel)
   • Stabilize {most_volatile_channel} (highest revenue volatility)
   • Model best practices from {most_stable_channel} (most stable performance)
   • Review strategy for underperforming channels: {', '.join(underperforming_channels[:3])}

PRODUCT PORTFOLIO:
   • Focus on Star Products for growth acceleration
   • Evaluate Dog Products for discontinuation or repositioning
   • Develop Question Mark products with targeted marketing
   • Maintain Cash Cow products for steady revenue

OPERATIONAL EFFICIENCY:
   • Optimize warehouse-product allocation based on specialization
   • Enhance sales team training for lower-performing teams
   • Implement dynamic pricing for high-discount-sensitivity products
""")


11. STRATEGIC RECOMMENDATIONS
------------------------------------------------------------

🎯 STRATEGIC RECOMMENDATIONS:

CHANNEL OPTIMIZATION:
   • Invest more in Wholesale (highest ROI channel)
   • Stabilize Wholesale (highest revenue volatility)
   • Model best practices from Online (most stable performance)
   • Review strategy for underperforming channels: In-Store, Online

PRODUCT PORTFOLIO:
   • Focus on Star Products for growth acceleration
   • Evaluate Dog Products for discontinuation or repositioning
   • Develop Question Mark products with targeted marketing
   • Maintain Cash Cow products for steady revenue

OPERATIONAL EFFICIENCY:
   • Optimize warehouse-product allocation based on specialization
   • Enhance sales team training for lower-performing teams
   • Implement dynamic pricing for high-discount-sensitivity products



In [13]:
print("\n12. PERFORMANCE MONITORING FRAMEWORK")
print("-" * 60)

# Key metrics for ongoing monitoring
monitoring_metrics = {
    'Revenue Growth (MoM)': 'Track monthly revenue changes by channel',
    'Market Share Stability': 'Monitor HHI and channel concentration',
    'Product Portfolio Balance': 'Maintain optimal Star/Cash Cow ratio',
    'Channel ROI': 'Ensure all channels exceed minimum ROI threshold',
    'Revenue Volatility': 'Keep CV below industry benchmarks',
    'Customer Acquisition Cost': 'Track cost efficiency by channel'
}

print("KEY METRICS FOR ONGOING MONITORING:")
for metric, description in monitoring_metrics.items():
    print(f"   • {metric}: {description}")

print("\n" + "="*80)
print("SALES PERFORMANCE ANALYSIS COMPLETE")
print("="*80)


12. PERFORMANCE MONITORING FRAMEWORK
------------------------------------------------------------
KEY METRICS FOR ONGOING MONITORING:
   • Revenue Growth (MoM): Track monthly revenue changes by channel
   • Market Share Stability: Monitor HHI and channel concentration
   • Product Portfolio Balance: Maintain optimal Star/Cash Cow ratio
   • Channel ROI: Ensure all channels exceed minimum ROI threshold
   • Revenue Volatility: Keep CV below industry benchmarks
   • Customer Acquisition Cost: Track cost efficiency by channel

SALES PERFORMANCE ANALYSIS COMPLETE
